In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

app = dash.Dash(__name__)
server = app.server

uniquelaunchsites = spacex_df['Launch Site'].unique().tolist()
lsites = [{'label': 'All Sites', 'value': 'All Sites'}] + [{'label': site, 'value': site} for site in uniquelaunchsites]

app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    dcc.Dropdown(id='site_dropdown', options=lsites, placeholder='Select a Launch Site here', searchable=True, value='All Sites'),
    html.Br(),
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload_slider',
        min=0,
        max=10000,
        step=1000,
        marks={i: f'{i} kg' for i in range(0, 10001, 1000)},
        value=[min_payload, max_payload]
    ),
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site_dropdown', component_property='value')]
)
def update_graph(site_dropdown):
    if site_dropdown == 'All Sites':
        df = spacex_df.groupby('class').size().reset_index(name='counts')
        df['class'] = df['class'].replace({1: 'Success', 0: 'Failure'})
        fig = px.pie(df, names='class', values='counts', hole=.4, title='Total Success vs Failure Launches')
    else:
        df = spacex_df[spacex_df['Launch Site'] == site_dropdown].groupby('class').size().reset_index(name='counts')
        df['class'] = df['class'].replace({1: 'Success', 0: 'Failure'})
        fig = px.pie(df, names='class', values='counts', hole=.4, title=f'Total Success vs Failure Launches for site {site_dropdown}')
    return fig

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site_dropdown', component_property='value'), Input(component_id="payload_slider", component_property="value")]
)
def update_scattergraph(site_dropdown, payload_slider):
    low, high = payload_slider
    if site_dropdown == 'All Sites':
        df = spacex_df
    else:
        df = spacex_df[spacex_df['Launch Site'] == site_dropdown]
    
    mask = (df['Payload Mass (kg)'] >= low) & (df['Payload Mass (kg)'] <= high)
    fig = px.scatter(
        df[mask], x="Payload Mass (kg)", y="class",
        color="Booster Version",
        size='Payload Mass (kg)',
        hover_data=['Payload Mass (kg)']
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=False, port=8052)  # Change the port number here


/tmp/ipykernel_9681/3094260940.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/ipykernel_9681/3094260940.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
